[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/insider-trading-sec-form4-python.ipynb)

# Track Insider Trading from SEC Form 4 with Python -- Free, No API Key

Use **edgartools** to track insider buying and selling from SEC Form 4 filings in Python -- completely free, no API key or paid subscription required. Form 4 is the most common insider trading disclosure: every purchase, sale, or option exercise by officers, directors, and 10% owners.

**What you'll learn:**
- Find insider trading filings for any public company
- Extract transaction details: who traded, how many shares, at what price
- Screen for large insider purchases and sales
- Export insider trading data to a pandas DataFrame

## Install edgartools

In [ ]:
!pip install -U edgartools

## Setup

The SEC requires all automated tools to identify themselves. Replace the email below with your own -- any valid email works.

In [ ]:
from edgar import *

# The SEC requires you to identify yourself (any email works)
set_identity("your.name@example.com")

## Get Insider Trading Data in 3 Lines

Pick a company, get its latest Form 4 filing, and extract the insider transaction:

In [ ]:
# Get the latest insider trade for Tesla
filing = Company("TSLA").get_filings(form="4")[0]
filing.obj()

## Extract the Transaction Summary

Use `get_ownership_summary()` to get structured data -- who traded, how many shares, the dollar value, and what type of transaction:

In [ ]:
form4 = filing.obj()
summary = form4.get_ownership_summary()

print(f"Insider:    {summary.insider_name}")
print(f"Position:   {summary.position}")
print(f"Company:    {summary.issuer}")
print(f"Activity:   {summary.primary_activity}")
print(f"Net change: {summary.net_change:,} shares")
print(f"Net value:  ${summary.net_value:,.0f}" if summary.net_value else "Net value:  N/A")

## View Individual Transactions

A single Form 4 filing can contain multiple transactions. Loop through them to see each one:

In [ ]:
for t in summary.transactions:
    print(f"{t.transaction_type}: code={t.code}, shares={t.shares_numeric:,}, price=${t.price_numeric}")

## Export to a pandas DataFrame

Convert any transaction summary to a DataFrame for further analysis:

In [ ]:
summary.to_dataframe(detailed=True)

## Screen for Large Insider Trades

Scan recent Form 4 filings to find the biggest insider purchases and sales. This is where insider trading data gets interesting:

In [ ]:
tsla = Company("TSLA")

for filing in tsla.get_filings(form="4").head(30):
    summary = filing.obj().get_ownership_summary()
    if summary.net_value and abs(summary.net_value) > 1_000_000:
        direction = "BUY" if summary.net_change > 0 else "SELL"
        print(f"{filing.filing_date}  {direction:4s}  {summary.insider_name:30s}  {summary.net_change:>12,} shares  ${abs(summary.net_value):>15,.0f}")

## Form 3: Initial Ownership Statements

Form 3 is filed when someone first becomes an insider. It shows their initial holdings:

In [ ]:
# Latest Form 3 for Tesla
form3_filing = tsla.get_filings(form="3")[0]
form3_filing.obj()

## Search Insider Filings Across All Companies

Use `get_filings()` to search Form 4 filings market-wide -- not just for one company:

In [ ]:
# All Form 4 filings filed today (or most recent trading day)
filings = get_filings(form="4")
print(f"Form 4 filings found: {len(filings)}")
filings.head(10)

## Why EdgarTools?

EdgarTools is free and open-source. Compare getting insider trading data:

**With edgartools (free, no API key):**
```python
summary = Company("TSLA").get_filings(form="4")[0].obj().get_ownership_summary()
print(summary.insider_name, summary.net_change, summary.net_value)
```

**Typical paid API approach ($50+/month, API key required):**
```python
from sec_api import QueryApi
api = QueryApi(api_key="YOUR_PAID_API_KEY")
query = {"query": {"query_string": {"query": 'ticker:TSLA AND formType:"4"'}}}
filings = api.get_filings(query)
# ... then parse XML yourself to extract transaction details
```

With edgartools, insider trading data is parsed into Python objects automatically -- no XML parsing, no API key, no monthly fee.

## Quick Reference

```python
from edgar import *
set_identity("your.name@example.com")

# Get Form 4 insider trading filings for a company
filings = Company("TSLA").get_filings(form="4")

# Parse a filing into a typed object
form4 = filings[0].obj()

# Get computed summary (net change, value, activity type)
summary = form4.get_ownership_summary()
summary.insider_name       # "Elon Musk"
summary.net_change         # +2,568,732 or -60,000
summary.net_value          # Dollar value of transaction
summary.primary_activity   # "Purchase", "Sale", "Grant/Award"

# Individual transactions within a filing
for t in summary.transactions:
    t.code                 # P=Purchase, S=Sale, A=Grant, M=Exercise
    t.shares_numeric       # Number of shares
    t.price_numeric        # Price per share

# Export to pandas
df = summary.to_dataframe(detailed=True)
```

## What's Next

You've learned how to track insider trading from SEC Form 4 filings with Python. Here are related tutorials:

- [Analyze Initial Insider Ownership (Form 3) in Depth](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Initial-Insider-Transactions.ipynb)
- [Search SEC Filings with Python](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/01_getting_started.ipynb)
- [Extract Financial Statements from SEC Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Viewing-Financial-Statements.ipynb)
- [Extract Earnings Releases from 8-K Filings](https://colab.research.google.com/github/dgunning/edgartools/blob/main/notebooks/Extract-Earnings-Releases.ipynb)

**Resources:**
- [EdgarTools Documentation](https://edgartools.readthedocs.io/)
- [GitHub Repository](https://github.com/dgunning/edgartools)
- [PyPI Package](https://pypi.org/project/edgartools/)

---

## Support EdgarTools

If you found this tutorial helpful, here are a few ways to support the project:

- **Star the repo** -- [github.com/dgunning/edgartools](https://github.com/dgunning/edgartools) -- it helps others discover edgartools
- **Visit edgartools.io** -- [edgartools.io](https://www.edgartools.io/) -- for more tutorials, articles, and updates
- **Report issues** -- found a bug or have a feature idea? [Open an issue](https://github.com/dgunning/edgartools/issues)
- **Share this notebook** -- know someone who works with SEC data? Send them the Colab link

*edgartools is free, open-source, and community-driven. No API key or paid subscription required.*